In [1]:
# 올바른 위치에서 택시 승객을 태우고 내리는 문제
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [2]:
# 환경 초기화
env.reset()
env.render()

# 각 칸에서의 경우의 수 : 6가지 status: south(0), north(1), east(2), west(3), pickup(4), dropoff(5)
print("Action Space : {}".format(env.action_space))

# 전체 경우의 수
print("State Space : {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space : Discrete(6)
State Space : Discrete(500)


In [3]:
# (taxi row, taxi column, passenger index, destination index)
# (택시의 행, 택시의 열, 승객위치, 목적지)
# R(0), G(1), Y(2), B(3)
state = env.encode(3, 1, 2, 3)
print("State:", state)
env.s = state
env.render()
# 벽에 부딪히면 -1 패널티 부여
# 승객이 없을 때는 노란색, 승객이 있을 때는 녹색
# R,G,Y,B : 가능한 픽업 및 목적지 위치
# 파란색 : 승객 픽업 장소

State: 331
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
# env.P 초기 보상 테이블
env.P[328]
# 이동할 때마다 -1, 승객승하차 -10

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [5]:
# 무작위 방식으로 처리
# 초기 상태 설정
env.s = 328
epochs = 0
penalties, reward = 0, 0

# 애니메이션 관련 정보를 저장하는 리스트
frames = []

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    # 보상이 -10이면 패널티 1 부여
    if reward == -10:
        penalties += 1
        
    # 애니메이션을 위하여 정보 기록
    frames.append({
    'frame': env.render(mode='ansi'),
    'state': state,
    'action': action,
    'reward': reward
    })
    epochs += 1
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 208
Penalties incurred: 62


In [7]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 208
State: 0
Action: 5
Reward: 20


In [8]:
# 강화학습으로 구현
import numpy as np

# q 테이블 초기화
q_table = np.zeros([env.observation_space.n,
env.action_space.n])

In [11]:
%%time
import random
from IPython.display import clear_output

# 하이퍼 파라미터
alpha = 0.1 # 학습률
gamma = 0.6 # 할인율
epsilon = 0.1

# 그래프 출력을 위한 리스트
all_epochs = []
all_penalties = []
for i in range(1, 100001):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False
    while not done:
        if random.uniform(0, 1) < epsilon:
            # 랜덤값이 0.1 미만이면 새로운 경로 탐색
            action = env.action_space.sample()

        else: # 그렇지 않으면 기존에 학습된 경로 선택
            # 학습된 q 테이블의 값 중 최대값
            action = np.argmax(q_table[state])
        next_state, reward, done, info = env.step(action)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        if reward == -10:
            penalties += 1
        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: total: 35.2 s
Wall time: 34.1 s


In [12]:
q_table[328]

array([ -2.41700296,  -2.27325184,  -2.41497306,  -2.36328201,
       -11.12182882, -11.29676903])

In [13]:
# Q-learning 종료 후의 성능 측정
total_epochs, total_penalties = 0, 0
episodes = 100

# 애니메이션을 위한 프레임 리스트
frames = []

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        if reward == -10:
            penalties += 1

        # 애니메이션을 위한 정보 기록
        frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
        )
        epochs += 1
    total_penalties += penalties
    total_epochs += epochs
    
print(f"Results after {episodes} episodes:")
print(f"에피소드당 평균 시간 간격: {total_epochs / episodes}")
print(f"에피소드당 평균 패널티: {total_penalties / episodes}")

Results after 100 episodes:
에피소드당 평균 시간 간격: 13.15
에피소드당 평균 패널티: 0.0


In [14]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1315
State: 0
Action: 5
Reward: 20
